In [85]:
pip install googletrans==3.1.0a0

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography.utils import int_from_bytes
  Preparing metadata (setup.py) ... done
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=21063 sha256=8127a116a8767c45fd0790d3f16b82bf678c2b6e2b2678ef56e8e9e8b9c2cbbb
  Stored in directory: /root/.cache/pip/wheels/26/36/33/1ead496ea4484c4df14c9d148c9e2676a1a2d821bc88d4f453
Successfully built googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 4.0.0rc1
    Uninstalling googletrans-4.0.0rc1:
      Successfully uninstalled googletrans-4.0.0rc1
Note: you may need to restart the kernel to use updated packages.


In [1]:
import nucleus_api, nucleus
import os
import csv
import json
import datetime
import time
from nucleus_api.rest import ApiException
import nucleus_api.api.nucleus_api as nucleus_helper
from pprint import pprint
import numpy as np
from pathlib import Path
from pymed import PubMed

configuration = nucleus_api.Configuration()
# configuration.host = 'localhost:5000'
configuration.host = 'http://3.91.117.194:5000/'
configuration.api_key['x-api-key'] = 'YTRgtK8T2H-hhR5NbmKW3w'

api_instance = nucleus_api.NucleusApi(nucleus_api.ApiClient(configuration))

/usr/local/lib/python3.6/dist-packages/pymongo/pyopenssl_context.py:26: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography.x509 import load_der_x509_certificate as _load_der_x509_certificate


In [2]:
dataset = 'PubMedDataset'
#create dataset

In [12]:
from flask import Flask, render_template
app = Flask(__name__)

@app.route('/')
def hello():
    return "pikachu"

@app.route('/home')
def home():
    # str | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\") (optional)
    query = ''
    custom_stop_words = [] # str | List of stop words. (optional)
    num_topics = 5 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
    metadata_selection = "" # dict | JSON object specifying metadata-based queries on the dataset, of type {"metadata_field": "selected_values"} (optional)
    #time_period = ""     # str | Time period selection. Choices: ["1M","3M","6M","12M","3Y","5Y",""] (optional)
    remove_redundancies = True # bool | If True, this option removes quasi-duplicates from the analysis. A quasi-duplicate would have the same NLP representation, but not necessarily the exact same text. (optional) (default True)

    #compute topics:
    payload = nucleus_api.Topics(dataset=dataset,                                
                                    query=query,                   
                                    custom_stop_words=custom_stop_words,     
                                    num_topics=num_topics,
                                    metadata_selection=metadata_selection,
                                    remove_redundancies=remove_redundancies)
        
    api_response = api_instance.post_topic_api(payload) 
    input_array = [["Topic", "Strength"]] + [[topic.keywords, float(topic.strength)] for topic in api_response.result.topics]
    return render_template('home.html', inputArray = input_array)

if __name__ == '__main__':
    app.run(host = "0.0.0.0", port=5002)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.17.0.2:5002/ (Press CTRL+C to quit)
172.17.0.1 - - [26/Feb/2023 18:23:08] "GET /home HTTP/1.1" 200 -


In [2]:
try:
    ip = get_ipython()
    running_notebook = True
except NameError:
    running_notebook = False
print(running_notebook)

True


In [5]:


pubmed = PubMed(tool="NLPLahont", email="21tuong.dc@vinuni.edu.vn")
results = pubmed.query("(e[Title]) AND ([DOI])", max_results=5)
for article in results:
    print (article.title)

Investigating the Impact of E-Cigarette Price and Tax on E-Cigarette Use Behavior.
Extracellular vesicles are conduits for E. coli heat-labile enterotoxin (LT) and the B-subunits of LT and cholera toxin in immune cell-to-cell communication.
Phenylethanoid glycosides extract from Cistanche deserticola ameliorates atherosclerosis in apolipoprotein E-deficient mice and regulates intestinal PPARγ-LXRα-ABCA1 pathway.
The Evaluation of Vitamin E and TiO
Essential Oils and Biological Activities of 


In [17]:
dataset = 'PubMedDataset'
#create dataset

In [3]:
nucleus.Dataset.delete_dataset(dataset)
pubmed = PubMed(tool="NLPLahont", email="21tuong.dc@vinuni.edu.vn")
results = pubmed.query("(e[Title]) AND ([DOI])", max_results=2000)
for article in results:
    newdocument = {}
    newdocument["title"] = article.title
    if type(article.publication_date) == str:
        newdocument["time"] = article.publication_date
    else:
        newdocument["time"] = article.publication_date.strftime("%Y-%m-%d")
    newdocument["content"] = article.abstract
    # load into dataset
    payload = nucleus_api.Appendjsonparams(dataset=dataset,
                                       document=newdocument)
    try:
        api_response = api_instance.post_append_json_to_dataset(payload)
        print(api_response.result)
    except ApiException as e:
        api_error = json.loads(e.body)
        print('ERROR:', api_error['message'])


INFO: Dataset named PubMedDataset not found
{'n_documents': 1, 'size': 1615}
{'n_documents': 2, 'size': 1512}
{'n_documents': 3, 'size': 1369}
{'n_documents': 4, 'size': 711}
{'n_documents': 5, 'size': 263}
{'n_documents': 6, 'size': 52}
{'n_documents': 7, 'size': 1187}
{'n_documents': 8, 'size': 1195}
{'n_documents': 9, 'size': 696}
{'n_documents': 10, 'size': 544}
{'n_documents': 11, 'size': 1336}
{'n_documents': 12, 'size': 1367}
{'n_documents': 13, 'size': 58}
{'n_documents': 14, 'size': 1239}
{'n_documents': 15, 'size': 220}
{'n_documents': 16, 'size': 428}
{'n_documents': 17, 'size': 1829}
{'n_documents': 18, 'size': 460}
ERROR: {'message': 'document must have nonempty content and title'}
{'n_documents': 19, 'size': 1128}
ERROR: {'message': 'document must have nonempty content and title'}
{'n_documents': 20, 'size': 747}
{'n_documents': 21, 'size': 525}
{'n_documents': 22, 'size': 15}
{'n_documents': 23, 'size': 1587}
ERROR: {'message': 'document must have nonempty content and ti

2023-02-26 22:45:26,314 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f07149a4128>: Failed to establish a new connection: [Errno 111] Connection refused',)': //datasets/append_json_to_dataset


{'n_documents': 1500, 'size': 1352}
{'n_documents': 1501, 'size': 403}
{'n_documents': 1502, 'size': 1349}
{'n_documents': 1503, 'size': 649}
ERROR: {'message': 'document must have nonempty content and title'}
{'n_documents': 1504, 'size': 1810}
ERROR: {'message': 'document must have nonempty content and title'}
{'n_documents': 1505, 'size': 1799}
ERROR: {'message': "document input must have 'content' field."}
{'n_documents': 1506, 'size': 1398}
{'n_documents': 1507, 'size': 1496}
{'n_documents': 1508, 'size': 1741}
{'n_documents': 1509, 'size': 1714}
ERROR: {'message': "document input must have 'content' field."}
{'n_documents': 1510, 'size': 1603}
{'n_documents': 1511, 'size': 286}
{'n_documents': 1512, 'size': 601}
{'n_documents': 1513, 'size': 1341}
ERROR: {'message': "document input must have 'content' field."}
{'n_documents': 1514, 'size': 862}
{'n_documents': 1515, 'size': 612}
{'n_documents': 1516, 'size': 1710}
{'n_documents': 1517, 'size': 1850}
{'n_documents': 1518, 'size': 

In [3]:
# str | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\") (optional)
query = ''
custom_stop_words = [] # str | List of stop words. (optional)
num_topics = 5 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
metadata_selection = "" # dict | JSON object specifying metadata-based queries on the dataset, of type {"metadata_field": "selected_values"} (optional)
#time_period = ""     # str | Time period selection. Choices: ["1M","3M","6M","12M","3Y","5Y",""] (optional)
remove_redundancies = True # bool | If True, this option removes quasi-duplicates from the analysis. A quasi-duplicate would have the same NLP representation, but not necessarily the exact same text. (optional) (default True)

#compute topics:
payload = nucleus_api.Topics(dataset=dataset,                                
                                 query=query,                   
                                 custom_stop_words=custom_stop_words,     
                                 num_topics=num_topics,
                                 metadata_selection=metadata_selection,
                                 remove_redundancies=remove_redundancies)
    
api_response = api_instance.post_topic_api(payload) 
#create a json file topics.json containing only topic keywords, weights and strength
topics = [{'keywords':topic.keywords, 
           'keywords_weight':topic.keywords_weight, 
           'strength':topic.strength} for topic in api_response.result.topics]
# print(json.dumps(topics, indent=4, ensure_ascii=False))
input_array = [["Topic", "Strength"]] + [[topic.keywords, topic.strength] for topic in api_response.result.topics]
# with open('demo_topics.json', 'w') as outfile:
# with open('/demo/demo_topics.json', 'w') as outfile:
#     json.dump(topics, outfile, indent=4, ensure_ascii=False
print(input_array.__str__())

[['Topic', 'Strength'], ['hepatitis virus;virus hev;hev infection;public health;viral hepatitis;acute viral;acute hepatitis;common acute', '0.2527'], ['covid pandemic;study aimed;nursing students;social media;health promoting;promoting behaviors;aimed investigate;learning covid', '0.3166'], ['cigarette users;current cigarette;users cigarette;non smokers;compared non;non users;tobacco cigarette;quit vaping', '0.2135'], ['cross sectional;sectional study;sectional survey;study conducted;survey conducted;sectional descriptive;conducted cross;descriptive cross', '0.1893'], ['level cigarette;low level;depressive symptoms;symptoms aces;greater depressive;cigarette greater;high level;stable low;cannabis stable;age cannabis;aces extraversion', '0.0277']]


In [6]:
topic_keywords = [topic[0].split(";") for topic in input_array]
print(topic_keywords)

[['Topic'], ['hepatitis virus', 'virus hev', 'hev infection', 'public health', 'viral hepatitis', 'acute viral', 'acute hepatitis', 'common acute'], ['covid pandemic', 'study aimed', 'nursing students', 'social media', 'health promoting', 'promoting behaviors', 'aimed investigate', 'learning covid'], ['cigarette users', 'current cigarette', 'users cigarette', 'non smokers', 'compared non', 'non users', 'tobacco cigarette', 'quit vaping'], ['cross sectional', 'sectional study', 'sectional survey', 'study conducted', 'survey conducted', 'sectional descriptive', 'conducted cross', 'descriptive cross'], ['level cigarette', 'low level', 'depressive symptoms', 'symptoms aces', 'greater depressive', 'cigarette greater', 'high level', 'stable low', 'cannabis stable', 'age cannabis', 'aces extraversion']]


In [8]:
from googletrans import Translator
translator = Translator(service_urls=['translate.googleapis.com'])
for topic in topic_keywords:
  translations = translator.translate(topic, dest='vi')
  topic_translated = [translation.text for translation in translations]
  print(topic_translated)


['Đề tài']
['virus viêm gan', 'virus với nhau', 'nhiễm trùng hev', 'sức khỏe cộng đồng', 'viêm gan siêu vi', 'virus cấp tính', 'viêm gan cấp', 'cấp tính thông thường']
['đại dịch covid', 'nghiên cứu nhằm mục đích', 'sinh viên điều dưỡng', 'truyền thông xã hội', 'tăng cường sức khỏe', 'thúc đẩy hành vi', 'nhằm mục đích điều tra', 'học covid']
['người sử dụng thuốc lá', 'thuốc lá hiện tại', 'người dùng thuốc lá', 'người không hút thuốc', 'so khong', 'người dùng không', 'thuốc lá thuốc lá', 'bỏ thuốc lá điện tử']
['mặt cắt ngang', 'nghiên cứu từng phần', 'khảo sát mặt cắt', 'nghiên cứu tiến hành', 'khảo sát tiến hành', 'mô tả mặt cắt', 'tiến hành chéo', 'chéo mô tả']
['cấp thuốc lá', 'cấp thấp', 'triệu chứng trầm cảm', 'triệu chứng', 'trầm cảm hơn', 'thuốc lá lớn hơn', 'cấp độ cao', 'thấp ổn định', 'cần sa ổn định', 'tuổi cần sa', 'át chủ bài']


In [58]:
query = ''
custom_stop_words = [] # str | List of stop words. (optional)
num_topics = 5 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
num_keywords = 4 # int | Number of keywords per topic that is extracted from the dataset. (optional) (default to 8)
summary_length = 3 # int | The maximum number of bullet points a user wants to see in each topic summary. (optional) (default to 6)
context_amount = 0 # int | The number of sentences surrounding key summary sentences in the documents that they come from. (optional) (default to 0)
num_docs = 20 # int | The maximum number of key documents to use for summarization. (optional) (default to 20)
excluded_docs = '' # str | List of document IDs that should be excluded from the analysis. Example, ["docid1", "docid2", ..., "docidN"]  (optional)
remove_redundancies = True # bool | If True, this option removes quasi-duplicates from the analysis. A quasi-duplicate would have the same NLP representation, but not necessarily the exact same text. (optional) (default True)
metadata_selection = ""
try:
    payload = nucleus_api.TopicSummaryModel	(dataset=dataset,                                
                                 query=query,
                                 summary_length = summary_length,
                                 num_keywords = 8,                 
                                 custom_stop_words=custom_stop_words,     
                                 num_topics=num_topics,
                                 metadata_selection=metadata_selection,
                                 remove_redundancies=remove_redundancies,
                                 time_period = "1M"
)
    api_response = api_instance.post_topic_summary_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:
    for i,res in enumerate(api_response.result):
        print('Topic', i, 'summary:')
        print('    Keywords:', res.keywords)
        for j in range(len(res.summary)):
            print(res.summary[j])
        print('---------------')

Topic 0 summary:
    Keywords: cross sectional;sectional study;study aimed;study conducted;aimed investigate;covid pandemic;nursing students;study carried
{'attribute': {'counts': None, 'time': 1677283200},
 'doc_id': '10148028714484385684',
 'sentences': "['This cross-sectional descriptive study was carried out from "
              "March to June 2021 in Afzalipour Hospital, Kerman, Iran.']",
 'title': 'Agreement for diagnosis of depression and anxiety between '
          'self-assessment with e-questionnaire and psychiatric telephone '
          'interview among post-COVID-19 patients.'}
{'attribute': {'counts': None, 'time': 1677196800},
 'doc_id': '527644652447543205',
 'sentences': "['This cross-sectional study investigates the experiences of "
              'students of Hawler Medical University (Erbil, Iraqi Kurdistan), '
              'exploring their e-learning experience and satisfaction when '
              'conventional classroom learning was suspended due to '
            

In [59]:
# update_period = 'd' # str | Frequency at which the historical anlaysis is performed. choices=["d","m","H","M"] (default to d)
# query = '' # str | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\") (optional)
# custom_stop_words = [] # str | List of stop words (optional)
# num_topics = 5 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
# #num_keywords = 8 # int | Number of keywords per topic that is extracted from the dataset. (optional) (default to 8)
# inc_step = 1 # int | Number of increments of the udpate period in between two historical computations. (optional) (default to 1)
# excluded_docs = [''] # str | List of document IDs that should be excluded from the analysis. Example, ["docid1", "docid2", ..., "docidN"]  (optional)
# custom_dict_file = {} # file | Custom sentiment dictionary JSON file. Example, {"field1": value1, ..., "fieldN": valueN} (optional)
# remove_redundancies = True # bool | If True, this option removes quasi-duplicates from the analysis. A quasi-duplicate would have the same NLP representation, but not necessarily the exact same text. (optional) (default True)

metadata_selection ="" # dict | JSON object specifying metadata-based queries on the dataset, of type {"metadata_field": "selected_values"} (optional)
time_period = ""     # str | Time period selection. Choices: ["1M","3M","6M","12M","3Y","5Y",""] (optional)
period_start = "2020-01-01 01:01:01" # str | Start date for the period to analyze within the dataset. Format: "YYYY-MM-DD HH:MM:SS"
period_end = "2024-01-01 01:01:01" # str | End date for the period to analyze within the dataset. Format: "YYYY-MM-DD HH:MM:SS"
api_response = None

# str | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\") (optional)
query = ''
custom_stop_words = [] # str | List of stop words. (optional)
num_topics = 5 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
metadata_selection = "" # dict | JSON object specifying metadata-based queries on the dataset, of type {"metadata_field": "selected_values"} (optional)
#time_period = ""     # str | Time period selection. Choices: ["1M","3M","6M","12M","3Y","5Y",""] (optional)
remove_redundancies = True # bool | If True, this option removes quasi-duplicates from the analysis. A quasi-duplicate would have the same NLP representation, but not necessarily the exact same text. (optional) (default True)


try:
    payload = nucleus_api.TopicHistoryModel(dataset=dataset,                                
                                 query=query,                   
                                 custom_stop_words=custom_stop_words,     
                                 num_topics=num_topics,
                                 metadata_selection=metadata_selection,
                                 remove_redundancies=remove_redundancies,
                                 time_period = "1M"
                                 )
    api_response = api_instance.post_topic_historical_analysis_api(payload)
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    print(e)

print('Printing historical metrics data...')

for i,res in enumerate(api_response.result):
    print('Topic', i, res.keywords)
    print('    Timestamps:', res.time_stamps)
    print('    Strengths:', res.strengths)
    print('    Consensuses:', res.consensuses)
    print('    Sentiments:', res.sentiments)
    print('----------------')
            

Printing historical metrics data...
Topic 0 cross sectional;sectional study;study aimed;study conducted;aimed investigate;covid pandemic;nursing students;study carried
    Timestamps: ['2023-02-02 00:00:00', '2023-02-05 00:00:00', '2023-02-11 00:00:00', '2023-02-12 00:00:00', '2023-02-16 00:00:00', '2023-02-23 00:00:00', '2023-02-24 00:00:00', '2023-02-26 00:00:00']
    Strengths: ['0.2955', '0.2533', '0.3343', '0.4425', '0.4206', '0.4085', '0.3481', '0.3196']
    Consensuses: ['0.5164', '0.6524', '0.6388', '0.4524', '0.4581', '0.5271', '0.852', '0.7383']
    Sentiments: ['0.0039', '-0.03', '-0.0612', '-0.0019', '-0.0226', '-0.045', '-0.0976', '-0.0894']
----------------
Topic 1 cigarette users;tobacco cigarette;users cigarette;non smokers;dual users;users dual;smokers tobacco;users tobacco
    Timestamps: ['2023-02-02 00:00:00', '2023-02-05 00:00:00', '2023-02-11 00:00:00', '2023-02-12 00:00:00', '2023-02-16 00:00:00', '2023-02-23 00:00:00', '2023-02-24 00:00:00', '2023-02-26 00:00:00